<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
#!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [4]:
num_classes = 2
image_resize = 224
batch_size_train = 100
batch_size_valid = 100

In [5]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize), 
    batch_size=batch_size_train, 
    class_mode="categorical")

Found 30001 images belonging to 2 classes.


In [7]:
valid_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize), 
    batch_size=batch_size_valid, 
    class_mode="categorical")

Found 9501 images belonging to 2 classes.


In [8]:
model = Sequential()
model.add(VGG16(include_top=False, pooling="avg", weights="imagenet"))
model.add(Dense(num_classes, activation='softmax'))
model.layers

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.layers[0].layers

In [10]:
model.layers[0].trainable = False

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
steps_per_epoch_train = len(train_generator)
steps_per_epoch_valid = len(valid_generator)
num_epochs = 1

In [14]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=100,
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
100/100 [==============================] - 6292s 63s/step - loss: 0.2544 - acc: 0.9230 - val_loss: 0.0842 - val_acc: 0.9788


In [15]:
model.save('classifier_vgg16_model.h5')

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
# START CODE BLOCK
# cos2file - takes an object from Cloud Object Storage and writes it to file on container file system.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: project object defined in project token
# data_path: the directory to write the file
# filename: name of the file in COS

import os
def cos2file(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    open( data_dir + '/' + filename, 'wb').write(p.get_file(filename).read())

# file2cos - takes file on container file system and writes it to an object in Cloud Object Storage.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: prooject object defined in project token
# data_path: the directory to read the file from
# filename: name of the file on container file system

import os
def file2cos(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    path_to_file = data_dir + '/' + filename
    if os.path.exists(path_to_file):
        file_object = open(path_to_file, 'rb')
        p.save_data(filename, file_object, set_project_asset=True, overwrite=True)
    else:
        print("file2cos error: File not found")
# END CODE BLOCK

In [18]:
file2cos(project, '/work', 'classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


In [19]:
cos2file(project, '/work', 'classifier_resnet_model.h5')
clf_resnet = keras.models.load_model("classifier_resnet_model.h5")

/opt/conda/envs/Python36/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [20]:
clf_vgg16 = keras.models.load_model("classifier_vgg16_model.h5")

Use the following cells to evaluate your models.

In [21]:
batch_size_test = 100

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize), 
    shuffle=False,
    batch_size=batch_size_test)

Found 500 images belonging to 2 classes.


In [22]:
steps_per_epoch_test = len(test_generator)
resnet_eval = clf_resnet.evaluate_generator(test_generator, steps=steps_per_epoch_test)

In [23]:
vgg16_eval = clf_vgg16.evaluate_generator(test_generator, steps=steps_per_epoch_test)

In [24]:
resnet_eval

[0.32275708557572214, 0.8460000038146973]

In [25]:
vgg16_eval

[0.08496658205986023, 0.9700000166893006]

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [34]:
y_prob = clf_vgg16.predict_generator(test_generator, steps=steps_per_epoch_test)

In [35]:
y_prob[:5]

array([[1.3709586e-02, 9.8629040e-01],
       [1.4604851e-04, 9.9985397e-01],
       [2.1234713e-02, 9.7876531e-01],
       [3.5210848e-02, 9.6478915e-01],
       [4.8495724e-04, 9.9951506e-01]], dtype=float32)

In [36]:
import numpy as np
y_pred = np.argmax(y_prob, axis=1)
y_pred[:5]

array([1, 1, 1, 1, 1])

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).